In [ ]:
import netgen.gui
from ngsolve import *
import netgen.geom2d as geom2dim

In [ ]:
geo = geom2dim.SplineGeometry()

vertices = [(0,0), (2,0), (2,1), (1,1), (1,2), (0,2)]
p1, p2, p3, p4, p5, p6 = [geo.AppendPoint(x,y) for x,y in vertices]
lines = [['line', start, end, 0] for start, end in [(p1, p2), (p2, p3), (p3, p4), (p4, p5), (p5, p6), (p6,p1)]]
[geo.Append(line) for line in lines]

number_of_vertices = 6
mesh_size = 1e-1

mesh = Mesh(geo.GenerateMesh(maxh=mesh_size))

grid_functions = []
L2Error = []
H1Error = []
Means = []
kmax = 8
ks = [i for i in range(1, kmax)]

In [ ]:
# gf_ref...reference solution
# H1-conforming finite element space (fes_ref)
fes_ref = H1(mesh, order=kmax, dirichlet=[0])

# Set trial and test function
u = fes_ref.TrialFunction()
v = fes_ref.TestFunction()

# Set right hand side
rhs = LinearForm(fes_ref)
#rh s += SymbolicLFI(1*v)
rhs += 1*v*dx

# The bilinear form
A = BilinearForm(fes_ref, symmetric=True)
A += grad(u)*grad(v)*dx

# Now assemble the system of equations
with TaskManager():
    rhs.Assemble()
    A.Assemble()

# Calculate the solution field (function)
gf_ref = GridFunction(fes_ref)
gf_ref.vec.data = A.mat.Inverse(fes_ref.FreeDofs(), inverse='sparsecholesky') * rhs.vec

# calculate mean value on / in Omega
mean_ref = Integrate(gf_ref, mesh)

# Draw reference solution
Draw(gf_ref)
Draw(grad(gf_ref), mesh, 'partialderivatives')

In [ ]:
for k in ks:
    # H1-conforming finite element space (fes)
    fes = H1(mesh, order=k, dirichlet=1)

    # Set trial and test function
    u = fes.TrialFunction()
    v = fes.TestFunction()

    # Set right hand side
    rhs = LinearForm(fes)
    #rh s += SymbolicLFI(1*v)
    rhs += 1*v*dx

    # The bilinear form
    A = BilinearForm(fes, symmetric=True)
    A += grad(u)*grad(v)*dx

    # Now assemble the system of equations
    with TaskManager():
        rhs.Assemble()
        A.Assemble()

    # Calculate the solution field (function)
    gf = GridFunction(fes)
    gf.vec.data = A.mat.Inverse(fes.FreeDofs(), inverse='sparsecholesky') * rhs.vec
    grid_functions.append(gf)

    # Append error (error^2 ?)
    diff_func = gf - gf_ref
    L2Error.append(sqrt(Integrate((diff_func)*(diff_func), mesh)))
    H1Error.append(sqrt(Integrate((diff_func)*(diff_func), mesh) + Integrate((grad(gf)-grad(gf_ref))*(grad(gf)-grad(gf_ref)),mesh)))

    # calculate mean value on / in Omega
    mean = Integrate(gf, mesh)
    Means.append(mean)

    print('-'*40)
    print("k=",k)
    print("L2-error:", L2Error[-1])
    print("H1-error:", H1Error[-1])
    print("mean: ", Means[-1])


Means.append(mean_ref)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

L2Error = np.array(L2Error)
H1Error = np.array(H1Error)
Means = np.array(Means)

plt.figure(1)
#plt.subplot(121)
plt.semilogy(ks, L2Error, '-pr', label='$\|L_2 \|$')
plt.semilogy(ks, H1Error, '-pb', label='$\|H_1 \|$')
plt.semilogy(np.arange(1,kmax+1), Means, '-pg', label='mean(u)')
plt.xlabel('k...order')
plt.ylabel('Relative error')
plt.title('Convergence of errors')
plt.grid()
plt.legend()
plt.savefig('/Users/peterholzner/Code/num-pde/Exercises/Ex1.4_errors.png')
plt.show()